## **Functional Programming**


**Pure function**

For the same entry you receive the same outputs

A pure function is a function that has the following properties:

- Its return value is the same for the same arguments
- Its evaluation has no side effects

https://www.youtube.com/watch?v=gZikNtcMOsk

In [1]:
def pure(x: Int, y: Int): Int = x + y

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.15:4043
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1586609026203)
SparkSession available as 'spark'


pure: (x: Int, y: Int)Int


In [3]:
println(pure(1, 2))
println(pure(1, 2))

3
3


### Impure function

For the same entry you receive different outputs

In [4]:
var acc: Int = 0
def impure(x: Int, y: Int): Int = {
    acc += 1
    x + y + acc
}

acc: Int = 0
impure: (x: Int, y: Int)Int


In [5]:
println(impure(1, 2))
println(impure(1, 2))

4
5


## **Inmutability**

In [7]:
val x: Int = 0 // error
x = 2

<console>: 25: error: reassignment to val

Read: **Why Functional Programming Matters? (John Hughes)**

"We conclude that since **modularity** is the key to successful programming, functional programming offers important advantages for software development."

## **Algebraic Data Types**
- Make a data type to work with int lists: a list can be empty or not (it has a head and a tail)
- sealed trait: define the only possible cases

In [8]:
sealed trait IList
class End() extends IList
class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


In [9]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
}
class End() extends IList
class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


the results are pointers

In [10]:
val xs: IList = new Cons(1, new Cons(2, new End())) // IList(1, 2)
xs.prepend(0) // IList(0, 1, 2)

xs: IList = Cons@19076c2f
res3: IList = Cons@c1baac4


##**Case classes**

They provide common utilities automatically: shorter constructor, toString method, copy method, etc.

In [11]:
case class Bicicleta(cadencia: Int, marcha: Int, velocidad: Int)

defined class Bicicleta


In [13]:
// instead of new Bicicleta(1, 2, 3) we can put 
val b = Bicicleta(1, 2, 3)
println(b.cadencia)
println(b.velocidad)
b.copy(velocidad = 4)

1
3


b: Bicicleta = Bicicleta(1,2,3)
res5: Bicicleta = Bicicleta(1,2,4)


* We can use it to make list creation simpler and to enable reading instance contents

In [16]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


In [18]:
val xs: IList = Cons(1, Cons(2, End()))
xs.prepend(0)

xs: IList = Cons(1,Cons(2,End()))
res8: IList = Cons(0,Cons(1,Cons(2,End())))


## Pattern Matching
We can use this technique to compare a value against a sequence of patterns

In [19]:
//switch of java
def myPm(x: Any): Int = x match {
    case i: Int => i
    case s: String => s.length
    case _ => -1
}

myPm: (x: Any)Int


In [23]:
println(myPm(5))
println((myPm("hi")))
println((myPm(3.0)))

5
2
-1


Pattern matching turns out to be really useful in combination with case classes (sum method)

In [24]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0 // defined before, for this reason you can not eliminate this
        case Cons(h, t) => h + t.sum //recursive
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


In [25]:
Cons(1, Cons(2, Cons(3, End()))).sum

res11: Int = 6


## **Lambda Expressions** 
* So far we've been working with methods (incr example)

In [26]:
def incr(x: Int): Int = x + 1
incr(0)

incr: (x: Int)Int
res12: Int = 1


But functions are first-class citizens (ugly lambda expressions)

In [27]:
val x: Int =0
val s: String = "hola"
val f: Function1[Int, Int] = new Function1[Int, Int] {
    def apply(x: Int): Int = x + 1
}

x: Int = 0
s: String = hola
f: Int => Int = <function1>


In [28]:
f.apply(0)

res13: Int = 1


We provide a new method, which mapss every element in the list

In [29]:
sealed trait IList {
    def prepend(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


In [30]:
val xs: IList = Cons(1, Cons(2, Cons(3, End())))
xs.map(f)

xs: IList = Cons(1,Cons(2,Cons(3,End())))
res14: IList = Cons(2,Cons(3,Cons(4,End())))


## **More**

An operator is just a method (concatenate & prepend)

In [36]:
sealed trait IList {
    def ::(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
    def ++(other: IList): IList = this match {
        case End() => other
        case Cons(h, t) => Cons(h, t ++ other)
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList) extends IList

defined trait IList
defined class End
defined class Cons


In [38]:
val xs: IList = Cons(1, Cons(2, End()))
xs ++ xs
println(xs.::(0))
println(0 :: xs)

Cons(0,Cons(1,Cons(2,End())))
Cons(0,Cons(1,Cons(2,End())))


xs: IList = Cons(1,Cons(2,End()))


It's possible to declare default arguments for parameters

In [39]:
sealed trait IList {
    def ::(h: Int): IList = new Cons(h, this)
    def sum: Int = this match {
        case End() => 0
        case Cons(h, t) => h + t.sum
    }
    def map(f: Function1[Int, Int]): IList = this match {
        case End() => End()
        case Cons(h, t) => Cons(f(h), t.map(f))
    }
    def ++(other: IList): IList = this match {
        case End() => other
        case Cons(h, t) => Cons(h, t ++ other)
    }
}
case class End() extends IList
case class Cons(head: Int, tail: IList = End()) extends IList

defined trait IList
defined class End
defined class Cons


In [40]:
Cons(1, End())
Cons(1)

res17: Cons = Cons(1,End())


Variadic methods are great for certain situations like creating lists

In [56]:
object IList{
    def apply(xs: Int*): IList =
      if (xs.isEmpty) End() else Cons(xs.head, apply(xs.tail:_*))
}

defined object IList
Companions must be defined together; you may wish to use :paste mode for this.


In [57]:
IList.apply(1, 2, 3)
IList(1, 2, 3)

res26: IList = Cons(1,Cons(2,Cons(3,End())))


Scala deploys syntax to simplify the creation of Lambda Expressions

In [58]:
val incr: Int => Int = (i: Int) => i + 1

incr: Int => Int = <function1>


In [59]:
incr(0)

res27: Int = 1


In [60]:
val incr: Int => Int = i => i + 1

incr: Int => Int = <function1>


In [61]:
val incr: Int => Int = _ + 1
incr(0)

incr: Int => Int = <function1>
res28: Int = 1


In [62]:
IList(1, 2, 3).map(_ + 1)
IList(1, 2, 3).map(_ * 5)

res29: IList = Cons(5,Cons(10,Cons(15,End())))


## More ADT examples
Optional Int

In [63]:
sealed trait IOption
case class INone() extends IOption
case class ISome(i: Int) extends IOption

defined trait IOption
defined class INone
defined class ISome


Either Error or Int

In [ ]:
sealed trait IEither
case class Left(err: Error) extends IEither
case class Right(i: Int) extends IEither

## Final result

In [64]:
// IList is an Algebraic Data Type, made of Cons and End
sealed trait IList {
    
  // Operators are available
  def ::(h: Int): IList = Cons(h, this)
    
  // Pattern matching is very handy to deal with ADTs
  def sum: Int = this match {
    case Cons(h, t) => h + t.sum
    case End() => 0
  }
    
  // Functions can be passed as parameters
  // `map` is a higher order function
  def map(f: Int => Int): IList = this match {
    case Cons(h, t) => Cons(f(h), t.map(f))
    case End() => End()
  }
    
  def ++(other: IList): IList = this match {
    case Cons(h, t) => Cons(h, t ++ other)
    case End() => other
  }
}

// Case classes mitigate common boilerplate
// Default parameters could be helpful
case class Cons(val head: Int, val tail: IList = End()) extends IList
case class End() extends IList

object IList {
  // Variadic arguments are idoneous as a list creator
  def apply(xs: Int*): IList = {
    if (xs.isEmpty) End()
    else Cons(xs.head, apply(xs.tail:_*)) 
  }
}

defined trait IList
defined class Cons
defined class End
defined object IList


## Takeaways
- Functional programming is programming with pure functions
- Algebraic data types are encoded as a "sum" of case clases
- Functions are treated as first-class citizens, which enables higher order functions
- Syntactic sugar is convenient to dulcify expressions
- Dotty has introduced many features towards the functional side
- This is just the beginning: type classes, DSLs, generic programming, etc.

## **Aplauses**

![Applause](misc/applause.gif)